In [1]:
from selenium import webdriver
import time
import pandas as pd
import re
from selenium.webdriver.common.by import By
#from datetime import datetime, date, timedelta
from selenium.webdriver.chrome.options import Options
import requests
import json

from webdriver_manager.chrome import ChromeDriverManager
import mysql
import mysql.connector
import os

import datetime

import psycopg2
import sshtunnel
from sshtunnel import SSHTunnelForwarder


from dotenv import load_dotenv
load_dotenv(".env")


print('ライブラリの読み込み完了')

    
def insert_data_bulk(df,cnx):
    insert_sql = """INSERT INTO schedule (都道府県, イベント日, 曜日, 店舗名, 取材名, 媒体名, 取材ランク,取得時間) values (%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur = cnx.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    print("Insert bulk data")



def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text



def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 


def login_scraping_site(area_name):
    from selenium.webdriver.chrome.service import Service
    from selenium import webdriver
    from chromedriver_py import binary_path # this will get you the path variable
    svc = Service(executable_path=binary_path)
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    browser = webdriver.Chrome(service=svc, options=options)
    url_login = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/login_form_mail"
    #admageを開く
    browser.get(url_login)
    browser.implicitly_wait(10)

    # id
    element = browser.find_element(By.NAME,"id")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_ID'))

    # pw
    element = browser.find_element(By.NAME,"pass")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_PW'))

    browser.implicitly_wait(10)
    element = browser.find_element(By.CLASS_NAME,"box_hole_view_report_input")
    element.click()
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/select_area"
    browser.get(url)
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/?area={area_name}"
    browser.get(url)
    browser.implicitly_wait(10)
    return browser


furture_syuzai_list_df = pd.DataFrame(index=[], columns=['都道府県','イベント日','店舗名','取材名','媒体名','取材ランク'])
#['hokkaido','tohoku','kanto','chubu','kansai','chugoku','sikoku','kyushu']
for area_name in ['hokkaido','tohoku','kanto','chubu','kansai','chugoku','sikoku','kyusyu']:
    browser = login_scraping_site(area_name)
    elements = browser.find_elements(By.CLASS_NAME,"mgn_serch_list_bottom")
    post_line_text(f'{area_name}の取材予定追加開始',os.getenv('LINE_TOKEN'))
    i = 0
    while True:
        browser.find_element(By.CLASS_NAME,"head_change_main").click()
        browser.implicitly_wait(10)
        if 'プレミアム会員登録' == browser.find_element(By.CLASS_NAME,"menu_child").text:
            browser = login_scraping_site(area_name)
        else:
            pass
        try:
            elements = browser.find_elements(By.CLASS_NAME,"mgn_serch_list_bottom")
            baitai_name = elements[i].text.split(' ')[0]
            print('baitai_name',baitai_name)
            if baitai_name == 'ホールナビ':
                i += 1
                continue
            baitai_name = baitai_name.replace('パチマガスロマガじゃ…','パチマガスロマガ').replace('パチンコ店長のホール…','パチンコ店長のホール攻略')
            if  ('県' in baitai_name) or ('府' in baitai_name) or ('東京都' in baitai_name) or ('北海道' in baitai_name):
                print('県がついているためパスを処理しました',baitai_name)
                i += 1
                continue

            print(baitai_name)
            elements[i].click()
            # if 'ホールナビ' in baitai_name:
            #     print(baitai_name)
            #     break
            num = 0
            while True:
                for syuzai_tenpo_data in browser.find_elements(By.CLASS_NAME,"osbox"):
                    tenpo_name = syuzai_tenpo_data.find_element(By.CLASS_NAME,"oslh2").text.replace('\n', '').replace(' ', '').replace('　', '')
                    #print(tenpo_name.text)
                    syuzai_date = syuzai_tenpo_data.find_element(By.CLASS_NAME,"oslmd").text
                    rank_and_syuzai_name = syuzai_tenpo_data.find_element(By.CLASS_NAME,"list_event_name").text
                    syuzai_rank = rank_and_syuzai_name.split('\n')[0]
                    syuzai_name = rank_and_syuzai_name.split('\n')[1]
                    prefectures = syuzai_tenpo_data.find_elements(By.CLASS_NAME,"oslha")[0].text
                    #print(baitai_name,syuzai_date ,syuzai_rank,syuzai_name,prefectures)#prefectures
                    record = pd.Series([prefectures,syuzai_date, tenpo_name,syuzai_name,baitai_name,syuzai_rank], index=furture_syuzai_list_df.columns)
                    record_df =  pd.DataFrame(record)
                    furture_syuzai_list_df = pd.concat([furture_syuzai_list_df,record_df.T])
                    #print(record)
                    #break
                    browser.implicitly_wait(10)
                if num >6:
                    break
                elif browser.find_element(By.CLASS_NAME,'navi_1_next').text == '次へ':
                    browser.find_element(By.CLASS_NAME,'navi_1_next').click()
                    num += 1
                    print('num',num)
                else:
                    break
            i += 1
            # time.sleep(1)
            #break
            #break
        except Exception as e:
            print('エラー理由',e)
            break
        # if i > 5:
        #     break
browser.quit()



ライブラリの読み込み完了


C:\Users\tsc95\AppData\Local\Temp\ipykernel_7436\174310579.py:66: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="C:\\Users\\tsc95\\.wdm\\drivers\\chromedriver\\win64\\116.0.5845.140\\chromedriver-win32\\chromedriver.exe",options=options)#ChromeDriverManager().install()


baitai_name 北海道
県がついているためパスを処理しました 北海道
baitai_name 北海道
県がついているためパスを処理しました 北海道
baitai_name ホールナビ
baitai_name 新台入替
新台入替
エラー理由 list index out of range


C:\Users\tsc95\AppData\Local\Temp\ipykernel_7436\174310579.py:66: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="C:\\Users\\tsc95\\.wdm\\drivers\\chromedriver\\win64\\116.0.5845.140\\chromedriver-win32\\chromedriver.exe",options=options)#ChromeDriverManager().install()


baitai_name 宮城県
県がついているためパスを処理しました 宮城県
baitai_name 福島県
県がついているためパスを処理しました 福島県
baitai_name 山形県
県がついているためパスを処理しました 山形県
baitai_name 岩手県
県がついているためパスを処理しました 岩手県
baitai_name 秋田県
県がついているためパスを処理しました 秋田県
baitai_name 青森県
県がついているためパスを処理しました 青森県
baitai_name 宮城県
県がついているためパスを処理しました 宮城県
baitai_name 福島県
県がついているためパスを処理しました 福島県
baitai_name 山形県
県がついているためパスを処理しました 山形県
baitai_name 岩手県
県がついているためパスを処理しました 岩手県
baitai_name 青森県
県がついているためパスを処理しました 青森県
baitai_name でちゃう！
でちゃう！
baitai_name アツ姫
アツ姫
baitai_name エースプロ
エースプロ
baitai_name スクープTV
スクープTV
baitai_name ホールサーチマン
ホールサーチマン
baitai_name ホールナビ
baitai_name 爆ガチ!
爆ガチ!
baitai_name リニューアルオープン
リニューアルオープン
baitai_name 新台入替
新台入替
エラー理由 list index out of range


...

baitai_name 広島県
県がついているためパスを処理しました 広島県
baitai_name 山口県
県がついているためパスを処理しました 山口県
baitai_name 岡山県
県がついているためパスを処理しました 岡山県
baitai_name 島根県
県がついているためパスを処理しました 島根県
baitai_name 鳥取県
県がついているためパスを処理しました 鳥取県
baitai_name 広島県
県がついているためパスを処理しました 広島県
baitai_name 山口県
県がついているためパスを処理しました 山口県
baitai_name 岡山県
県がついているためパスを処理しました 岡山県
baitai_name 島根県
県がついているためパスを処理しました 島根県
baitai_name ホールナビ
baitai_name 旧イベ
旧イベ
num 1
num 2
num 3
baitai_name 新台入替
新台入替
エラー理由 list index out of range


C:\Users\tsc95\AppData\Local\Temp\ipykernel_7436\174310579.py:66: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="C:\\Users\\tsc95\\.wdm\\drivers\\chromedriver\\win64\\116.0.5845.140\\chromedriver-win32\\chromedriver.exe",options=options)#ChromeDriverManager().install()


baitai_name 愛媛県
県がついているためパスを処理しました 愛媛県
baitai_name 香川県
県がついているためパスを処理しました 香川県
baitai_name 高知県
県がついているためパスを処理しました 高知県
baitai_name 徳島県
県がついているためパスを処理しました 徳島県
baitai_name 愛媛県
県がついているためパスを処理しました 愛媛県
baitai_name 香川県
県がついているためパスを処理しました 香川県
baitai_name 徳島県
県がついているためパスを処理しました 徳島県
baitai_name ホールナビ
baitai_name 旧イベ
旧イベ
num 1
num 2
baitai_name 新台入替
新台入替
エラー理由 list index out of range


C:\Users\tsc95\AppData\Local\Temp\ipykernel_7436\174310579.py:66: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="C:\\Users\\tsc95\\.wdm\\drivers\\chromedriver\\win64\\116.0.5845.140\\chromedriver-win32\\chromedriver.exe",options=options)#ChromeDriverManager().install()


baitai_name 福岡県
県がついているためパスを処理しました 福岡県
baitai_name 佐賀県
県がついているためパスを処理しました 佐賀県
baitai_name 長崎県
県がついているためパスを処理しました 長崎県
baitai_name 熊本県
県がついているためパスを処理しました 熊本県
baitai_name 大分県
県がついているためパスを処理しました 大分県
baitai_name 宮崎県
県がついているためパスを処理しました 宮崎県
baitai_name 鹿児島県
県がついているためパスを処理しました 鹿児島県
baitai_name 沖縄県
県がついているためパスを処理しました 沖縄県
baitai_name 福岡県
県がついているためパスを処理しました 福岡県
baitai_name 佐賀県
県がついているためパスを処理しました 佐賀県
baitai_name 長崎県
県がついているためパスを処理しました 長崎県
baitai_name 熊本県
県がついているためパスを処理しました 熊本県
baitai_name 大分県
県がついているためパスを処理しました 大分県
baitai_name 宮崎県
県がついているためパスを処理しました 宮崎県
baitai_name 鹿児島県
県がついているためパスを処理しました 鹿児島県
baitai_name 沖縄県
県がついているためパスを処理しました 沖縄県
baitai_name DMMぱちタウン
DMMぱちタウン
num 1
baitai_name PS記念日調査
PS記念日調査
baitai_name その他取材・企画
その他取材・企画
baitai_name でちゃう！
でちゃう！
baitai_name スロパチステーション
スロパチステーション
baitai_name ホールナビ
baitai_name 頂-ITADAKI-
頂-ITADAKI-
baitai_name 鬼姫
鬼姫
baitai_name 旧イベ
旧イベ
num 1
num 2
num 3
num 4
num 5
num 6
num 7
baitai_name リニューアルオープン
リニューアルオープン
baitai_name 新台入替
新台入替
num 1
エラー理由 list index out of ra

In [8]:
# try:
pattern = '東京都|北海道|(京都|大阪)府|.{2,3}県'
# 都道府県を抽出する
furture_syuzai_list_df = furture_syuzai_list_df[furture_syuzai_list_df['都道府県'] != '']
# furture_syuzai_list_df['都道府県']=furture_syuzai_list_df['都道府県'].apply(lambda x:re.match(pattern,x).group())
prefectures_list = []
for adress in furture_syuzai_list_df['都道府県']:
    adress = adress.split(']')[-1]
    try:
        #print(re.match(pattern,adress).group())
        prefectures_list.append(re.match(pattern,adress).group())
    except:
        prefectures_list.append('情報なし')
furture_syuzai_list_df['都道府県'] = prefectures_list
furture_syuzai_list_df_1 = furture_syuzai_list_df
furture_syuzai_list_df_1 = pd.concat([furture_syuzai_list_df_1, furture_syuzai_list_df_1['イベント日'].str.split('(', expand=True)], axis=1).drop('イベント日', axis=1)
furture_syuzai_list_df_1.rename(columns={0: 'イベント日', 1: '曜日'}, inplace=True)
furture_syuzai_list_df_1['曜日'] = furture_syuzai_list_df_1['曜日'].map(lambda x:x.replace(')',''))
furture_syuzai_list_df_1['イベント日'] = pd.to_datetime(furture_syuzai_list_df_1['イベント日'])
furture_syuzai_list_df_1 = furture_syuzai_list_df_1 [['都道府県','イベント日','曜日',	'店舗名','取材名','媒体名','取材ランク']]
furture_syuzai_list_df_1.drop_duplicates(keep='first', inplace=True)
#furture_syuzai_list_df_1 = furture_syuzai_list_df_1[~furture_syuzai_list_df_1['媒体名'] == 'ホールナビ']
furture_syuzai_list_df_1
# except Exception as e:
#     print('エラー理由',e)

,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク
0,北海道,2023-10-04,水,ジェイ.プレスネオ,新台入替(P-WORLD引用),新台入替,・
0,北海道,2023-10-04,水,パーラー太陽北24条店,新台入替(P-WORLD引用),新台入替,・
0,北海道,2023-10-04,水,パーラー太陽新琴似店,新台入替(P-WORLD引用),新台入替,・
0,北海道,2023-10-04,水,パーラー太陽苗穂本町店,新台入替(P-WORLD引用),新台入替,・
0,北海道,2023-10-04,水,ジェイ.プレスパレット,新台入替(P-WORLD引用),新台入替,・
...,...,...,...,...,...,...,...
0,熊本県,2023-10-05,木,夢屋天草店,新台入替(P-WORLD引用),新台入替,・
0,大分県,2023-10-05,木,パーラージェイズ,新台入替(P-WORLD引用),新台入替,・
0,福岡県,2023-10-05,木,PSJBIGBOM新宮店,新台入替(P-WORLD引用),新台入替,・
0,福岡県,2023-10-05,木,ワンダーランド八江店,新台入替(P-WORLD引用),新台入替,・


In [ ]:




server = sshtunnel.SSHTunnelForwarder((os.getenv('SSH_USERNAME'), 10022), 
    ssh_username="pachislot777", 
    ssh_private_key_password='akasaka2', 
    ssh_pkey="akasaka2.key", 
    remote_bind_address=("mysql8055.xserver.jp", 3306 )) 
# SSH接続確認


# 現在時刻
today = datetime.datetime.now()
print(today)
#[結果] 2021-08-23 07:12:20.806648
# 1日後
today_str:str = today.strftime('%Y-%m-%d')
eight_days_after:str = (today + datetime.timedelta(days=8)).strftime('%Y-%m-%d')
yesterday:str = (today + datetime.timedelta(days=-2)).strftime('%Y-%m-%d')
#### Create dataframe from resultant table ####

server.start()

print(f"local bind port: {server.local_bind_port}")
# データベース接続
cnx = mysql.connector.connect(
    host="localhost", 
    port=server.local_bind_port, 
    user=os.getenv('WORDPRESS_DB_ID'), 
    password=os.getenv('DB_PASSWORD'), 
    database=os.getenv('WORDPRESS_DB_NAME'), 
    charset='utf8',
    use_pure=True
    )

# 接続確認
print(f"sql connection status: {cnx.is_connected()}")
cursor = cnx.cursor()

prefectures ="("
prefecture_list = ['東京都','千葉県','埼玉県','神奈川県','茨城県','群馬県','栃木県']
for i,text in enumerate(prefecture_list):
    if i == (len(prefecture_list) -1 ):
        prefectures += f"'{text}'"
    else:
        prefectures += f"'{text}',"
prefectures += ')'
print(prefectures)

concat_df = furture_syuzai_list_df_1
concat_df['イベント日'] = concat_df['イベント日'].astype(str)   
concat_df = concat_df[~concat_df.duplicated(keep=False)]
concat_df['取得時間'] = today_str
concat_df['取得時間'] = concat_df['取得時間'].astype(str)   
concat_df

#UPSERTするために現在のデータを削除
sql = f"""
        DELETE
        FROM schedule
        WHERE イベント日 BETWEEN '{today_str}' AND '{eight_days_after}'
        AND  都道府県 in {prefectures}
        """
print(sql)
cursor.execute(sql)
cnx.commit()

insert_data_bulk(concat_df ,cnx)
cnx.commit()
server.stop()
post_line_text(f'{len(concat_df)}件のxサーバーへの全国の取材予定追加おわり',os.getenv('LINE_TOKEN'))
print(f'{len(concat_df)}件の全国の取材予定追加おわり')

## Postgresへのデータ登録

def insert_data_bulk(df,cnx):
    insert_sql = """INSERT INTO schedule (id,都道府県, イベント日, 曜日, 店舗名, 取材名, 媒体名, 取材ランク,取得時間) values (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur = cnx.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    print("Insert bulk data")

concat_df = furture_syuzai_list_df_1
concat_df['イベント日'] = concat_df['イベント日'].astype(str)   
concat_df.drop_duplicates(keep='first', inplace=True)
concat_df['取得時間'] = today_str
concat_df['取得時間'] = concat_df['取得時間'].astype(str)   

concat_df['id'] = 0
cols = concat_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
concat_df = concat_df[cols] 

users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
port = 5432        # DBが稼働しているポート番号(適宜変更)

# PostgreSQLへ接続
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

# PostgreSQLにデータ登録
cursor = conn.cursor()

#UPSERTするために現在のデータを削除
sql = f"""
        DELETE
        FROM schedule
        WHERE イベント日 BETWEEN '{today_str}' AND '{eight_days_after}'
        AND  都道府県 in {prefectures}
        """
print(sql)
cursor.execute(sql)
conn.commit()

insert_data_bulk(concat_df ,conn)
conn.commit()
post_line_text(f'{len(concat_df)}件のpostgresへの全国の取材予定追加おわり',os.getenv('LINE_TOKEN'))
print(f'{len(concat_df)}件の全国の取材予定追加おわり')

# except Exception as e:
#     print('エラー理由',e)
#     post_line_text(f'エラー{e}',os.getenv('LINE_TOKEN'))
#     #break

In [11]:
post_line_text(f'ああああ',os.getenv('LINE_TOKEN'))

In [254]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os 
import time
import datetime
import sys
from pprint import pprint
# .envファイルから環境変数を読み込む
from dotenv import load_dotenv
load_dotenv()


True

In [49]:
url = os.getenv('KOUYAKU_SCRAPIN_URL') #環境変数からURLを取得
response = requests.get(url) #データを取得
soup = BeautifulSoup(response.text, 'html.parser')

In [167]:
kiji_list = soup.find_all( class_="blogcard-type bct-detail")
#aタグの中のhref属性を取得
kiji_url_list = [kiji.find("a").get("href") for kiji in kiji_list]


len(kiji_url_list)

60

In [55]:
#環境変数からURLを取得
response = requests.get(kiji_url) #データを取得
groupby_media_soup = BeautifulSoup(response.text, 'html.parser')
#クラス名のないdivタグを取得


In [61]:
#h2タグを取得
for h2_tag in groupby_media_soup.find_all("h2"):
    if 'のイベント公約まとめ' in h2_tag.text:
        print(h2_tag.text)
        media_name = h2_tag.text.replace('のイベント公約まとめ', '')
        

鉄カバ(ヒッポ)のイベント公約まとめ
パチンコ・スロット総合サイト hippohippoはパチンコ・パチスロのまとめサイトです。hippo777.com


In [88]:
kiji_url_list

['https://manmai.club/?p=18979']

In [92]:
kiji_url_list

['https://manmai.club/?p=21798']

In [110]:
kouyaku_df_type_1 = pd.DataFrame(index=[],columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])
error_media_name_and_syuzai_name_dict = {}
for kiji_url in  kiji_url_list:
    #time.sleep(1)
    response = requests.get(kiji_url) #データを取得
    groupby_media_soup = BeautifulSoup(response.text, 'html.parser')
    #h2タグからメディア名を取得
    for h2_tag in groupby_media_soup.find_all("h2"):
        if 'イベント公約まとめ' in h2_tag.text:
            print(h2_tag.text)
            media_name = h2_tag.text.replace('のイベント公約まとめ', '')
    #クラス名のないdivタグを取得
    for div_elem in groupby_media_soup.find_all('div'):
        #print(div_elem.get('class'))
        if div_elem.get('class') == None and div_elem.find('h3') != None:
            #print(div_elem.text)
            #h3要素を取得
            #print(div_elem.h3.next_sibling)
            syuzai_name = div_elem.h3.text.replace('の公約', '')
            try:
                image_url = div_elem.h3.next_sibling.img.get('src')
                kouyaku_text = div_elem.h3.next_sibling.next_sibling.text.replace('公約　', '')
                kouyaku_df_type_1 = pd.concat([kouyaku_df_type_1, pd.DataFrame([[media_name,syuzai_name, image_url, kouyaku_text]], columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])])
            except:
                try:
                    image_url = ''
                    kouyaku_text = div_elem.h3.next_sibling.text.replace('公約　', '')
                    kouyaku_df_type_1 = pd.concat([kouyaku_df_type_1, pd.DataFrame([[media_name,syuzai_name, image_url, kouyaku_text]], columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])])
                except Exception as e:
                    error_media_name_and_syuzai_name_dict[media_name + syuzai_name] = e
                    continue
            #print(syuzai_name,kouyaku_text)
            

    #break

鉄カバ(ヒッポ)のイベント公約まとめ
ドリスロのイベント公約まとめ
スロットライブ【RED BLUE】のイベント公約まとめ
ななライズのイベント公約まとめ
肉はこう焼く!(エースプロ)のイベント公約まとめ
魚は焼かない！(エースプロ)のイベント公約まとめ
共闘戦線(RISING OUT)(エースプロ)のイベント公約まとめ
虹なな(エースプロ)のイベント公約まとめ
エースプロまとめサイトのイベント公約まとめ
APチャンネルのイベント公約まとめ
鬼姫のイベント公約まとめ
パチとスロは友を呼ぶのイベント公約まとめ
あかだ主任のガチンコホールのイベント公約まとめ
りゅーこのイベント公約まとめ
ホールサーチマンのイベント公約まとめ
五神獣のイベント公約まとめ
五神星のイベント公約まとめ
野菜はこう焼く！のイベント公約まとめ
BASHtv(バッシュTV)のイベント公約まとめ
PACHINKO&SLOT アナコンダのイベント公約まとめ
Aims (エイムズ)のイベント公約まとめ
リアルガチなホールチェックのイベント公約まとめ
飛蔵(とぶぞう)のパチンコ道場のイベント公約まとめ
イナズマ店長のイベント公約まとめ
パチエースのイベント公約まとめ
裏店長ミリオネアのイベント公約まとめ
そにま(Sonic Map)のイベント公約まとめ
アツモリ！のイベント公約まとめ
かみちゃま・かみたまのイベント公約まとめ


In [198]:
'鉄カバ(ヒッポ)'   in kouyaku_df_type_1['media_name'].unique()

True

In [119]:
list(kouyaku_df_type_1['media_name'].unique())

['鉄カバ(ヒッポ)',
 'ドリスロ',
 'ななライズ',
 '肉はこう焼く!(エースプロ)',
 'APチャンネル',
 '鬼姫',
 'パチとスロは友を呼ぶ',
 'あかだ主任のガチンコホール',
 'りゅーこ',
 'ホールサーチマン',
 '五神獣',
 '五神星',
 '野菜はこう焼く！',
 'BASHtv(バッシュTV)',
 'PACHINKO&SLOT アナコンダ',
 'Aims (エイムズ)',
 'リアルガチなホールチェック',
 '飛蔵(とぶぞう)のパチンコ道場',
 'イナズマ店長',
 'パチエース',
 '裏店長ミリオネア',
 'そにま(Sonic Map)',
 'アツモリ！',
 'かみちゃま・かみたま']

In [131]:
kiji_url_list[3]

'https://manmai.club/?p=21798'

In [188]:
kouyaku_df_type_2 = pd.DataFrame(index=[],columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])
error_media_name_and_syuzai_name_dict = {}
for kiji_url in  kiji_url_list:
    #time.sleep(1)
    response = requests.get(kiji_url) #データを取得
    groupby_media_soup = BeautifulSoup(response.text, 'html.parser')
    #h2タグからメディア名を取得
    for h2_tag in groupby_media_soup.find_all("h2"):
        if 'イベント公約まとめ' in h2_tag.text:
            media_name = h2_tag.text.replace('のイベント公約まとめ', '').replace('イベント公約まとめ', '')
    # if media_name in list(kouyaku_df_type_1['media_name'].unique()):
    #     print(media_name,'は既に取得済みです')
    #     continue
    #クラス名のないdivタグを取得
    for div_elem in groupby_media_soup.find_all('h3'):
        if 'toc' not in str(div_elem):
            #print('toc' not in str(div_elem))
            continue
        syuzai_name = div_elem.text.rstrip('の公約').rstrip('公約').lstrip('【Sランク】').lstrip('【Aランク】').lstrip('【Bランク】').lstrip('【Cランク】')
        try:
            image_url = div_elem.next_sibling.img.get('src')
            kouyaku_text = div_elem.next_sibling.next_sibling.text.replace('公約　', '')
            kouyaku_df_type_2 = pd.concat([kouyaku_df_type_2, pd.DataFrame([[media_name,syuzai_name, image_url, kouyaku_text]], columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])])
        except:
            try:
                image_url = ''
                kouyaku_text = div_elem.next_sibling.text.replace('公約　', '')
                kouyaku_df_type_2 = pd.concat([kouyaku_df_type_2, pd.DataFrame([[media_name,syuzai_name, image_url, kouyaku_text]], columns=['media_name' ,'syuzai_name', 'image_url', 'kouyaku_text'])])
            except Exception as e:
                error_media_name_and_syuzai_name_dict[media_name + syuzai_name] = e
                continue
        #print(syuzai_name,kouyaku_text)
        

    #break

In [ ]:
sorted(list(kouyaku_df_type_2['kouyaku_text'].unique()))


In [ ]:
kouyaku_df_type_2['media_name'].unique()

In [ ]:
kouyaku_df_type_2

In [ ]:
'【Sランク】スロパチステーション潜入取材'.lstrip('【Sランク】').lstrip('【Aランク】').lstrip('【Bランク】').lstrip('【Cランク】')

In [ ]:
kouyaku_df_type_2[kouyaku_df_type_2['media_name'].str.contains('スロパチ')]

In [ ]:
kiji_url_list

In [75]:
error_media_name_and_syuzai_name_dict

{'BASHtv(バッシュTV)源界突破パチンコレポート': AttributeError("'NoneType' object has no attribute 'text'"),
 'BASHtv(バッシュTV)【Sランク】爆ガチ広告': AttributeError("'NoneType' object has no attribute 'text'")}

In [76]:
kouyaku_df[kouyaku_df['media_name'].str.contains('スロパチ')]

,media_name,syuzai_name,image_url,kouyaku_text


In [85]:
kouyaku_df['media_name'].unique()

array(['鉄カバ(ヒッポ)', 'ドリスロ', 'ななライズ', '肉はこう焼く!(エースプロ)', 'APチャンネル', '鬼姫',
       'パチとスロは友を呼ぶ', 'あかだ主任のガチンコホール', 'りゅーこ', 'ホールサーチマン', '五神獣', '五神星',
       '野菜はこう焼く！', 'BASHtv(バッシュTV)', 'PACHINKO&SLOT アナコンダ', 'Aims (エイムズ)',
       'リアルガチなホールチェック', '飛蔵(とぶぞう)のパチンコ道場', 'イナズマ店長', 'パチエース', '裏店長ミリオネア',
       'そにま(Sonic Map)', 'アツモリ！', 'かみちゃま・かみたま'], dtype=object)

In [255]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
port = 5432        # DBが稼働しているポート番号(適宜変更)

# PostgreSQLへ接続
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

# カーソルを取得する
cursor = conn.cursor()

#自動コミットモードにする

conn.autocommit = True
conn.autocommit

True

In [256]:
cursor.execute('''SELECT *
               FROM pledge
               ''')
cols = [col.name for col in cursor.description]
pledge_table_df = pd.DataFrame(cursor.fetchall(),columns=cols)
pledge_table_df

,id,取材名,媒体名,公約内容,取得時間
0,100,妖-あやかし-(ヨースケ),1game,None,2023-07-23 07:39:15.857590
1,100,1GAMEホール取材 オブジョイトイ,1game,None,2023-07-23 07:39:16.372895
2,100,1GAMEホール取材 むつかけ,1game,None,2023-07-23 07:39:16.887209
3,100,来店実戦_BASHtv,BASHtv,None,2023-07-23 07:39:17.411343
4,100,自由打夢,BASHtv,None,2023-07-23 07:39:17.924061
...,...,...,...,...,...
742,0,Bakyun!スケールL,パチスロ必勝本,None,2023-08-14 07:34:58.616657
743,0,スロッター×スロッター30,パチンコ店長のホール攻略,None,2023-08-14 07:34:59.140888
744,0,アルティメットちゅんげー,天草ヤスヲ,None,2023-08-14 07:35:00.775477
745,0,裏寿司(かっぱ巻き),裏寿司解析,None,2023-08-14 07:35:01.330241


In [214]:
len(kouyaku_df_type_2['media_name'].unique())

59

In [251]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
def syuzai_df():
    global df, gs, SPREADSHEET_KEY
    SCOPE = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('sercret/twitteranalytics-jsonsercretkey.json', SCOPE)
    gs = gspread.authorize(credentials)
    SPREADSHEET_KEY = '1WxS9gZtK1vusQr-iHYt65HggSNkbOmclGzX6ipkesUg'
    worksheet = gs.open_by_key(SPREADSHEET_KEY).worksheet('master')
    df = pd.DataFrame(worksheet.get_all_values())
    df.columns = list(df.loc[0, :])
    df.drop(0, inplace=True)
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    return df

In [281]:
read_spreadsheet_syuzai_df

,syuzai_name,baitai_name,kouyaku
0,金色バナー,裏店長ミリオネア,・スロットコーナーで10.5割以上の営業を実施? ・3台以上構成の2機種以上に全台設定⑤⑥を投入?
1,虹色バナー,裏店長ミリオネア,・3台以上構成の2機種以上に全台設定⑤⑥を投入?・4台以上機種の1台以上に設定⑥投入? ・バ...
2,蛇色バナー,裏店長ミリオネア,・20円スロットの10％以上に設定⑤⑥投入? ・5台以上並びの1ヶ所以上に設定⑤⑥投入? 各...
3,黒色バナー,裏店長ミリオネア,・20円スロットの6％以上に設定⑤⑥投入? ・3台以上並びの1ヶ所以上に設定⑤⑥投入?
4,緑色バナー,裏店長ミリオネア,3台以上機種の1機種以上に設定⑤⑥投入?
...,...,...,...
4125,開眼REPORT,APチャンネル,バジリスクが対象 ボーダー以上
4126,英雄REPORT,APチャンネル,PF蒼穹のファフナー2が対象 ボーダー以上
4127,鬼がかりReport,APチャンネル,リゼロシリーズが対象 ボーダー以上
4128,頂上決戦!,APチャンネル,大都技研、サミー、ユニバーサル系機種の設置台数の10％に設定④⑤⑥


In [257]:
read_spreadsheet_syuzai_df = syuzai_df()
read_spreadsheet_syuzai_df

,syuzai_name,baitai_name,kouyaku
0,金色バナー,裏店長ミリオネア,・スロットコーナーで10.5割以上の営業を実施? ・3台以上構成の2機種以上に全台設定⑤⑥を投入?
1,虹色バナー,裏店長ミリオネア,・3台以上構成の2機種以上に全台設定⑤⑥を投入?・4台以上機種の1台以上に設定⑥投入? ・バ...
2,蛇色バナー,裏店長ミリオネア,・20円スロットの10％以上に設定⑤⑥投入? ・5台以上並びの1ヶ所以上に設定⑤⑥投入? 各...
3,黒色バナー,裏店長ミリオネア,・20円スロットの6％以上に設定⑤⑥投入? ・3台以上並びの1ヶ所以上に設定⑤⑥投入?
4,緑色バナー,裏店長ミリオネア,3台以上機種の1機種以上に設定⑤⑥投入?
...,...,...,...
4126,英雄REPORT,APチャンネル,PF蒼穹のファフナー2が対象 ボーダー以上
4127,鬼がかりReport,APチャンネル,リゼロシリーズが対象 ボーダー以上
4128,頂上決戦!,APチャンネル,大都技研、サミー、ユニバーサル系機種の設置台数の10％に設定④⑤⑥
4129,1日回胴録ハンチョウ石神収録 ガット石神,1GAME,来店実践


In [ ]:
pledge_table_df

In [266]:
read_spreadsheet_syuzai_df = read_spreadsheet_syuzai_df[(read_spreadsheet_syuzai_df['baitai_name'] !='未調査') & (read_spreadsheet_syuzai_df['kouyaku'] !='未調査')]
pledge_table_df.rename(columns={'取材名':'syuzai_name','公約内容':'kouyaku','媒体名':'baitai_name'}, inplace=True)

In [267]:
pledge_table_df

,id,syuzai_name,baitai_name,kouyaku,取得時間
0,100,妖-あやかし-(ヨースケ),1game,None,2023-07-23 07:39:15.857590
1,100,1GAMEホール取材 オブジョイトイ,1game,None,2023-07-23 07:39:16.372895
2,100,1GAMEホール取材 むつかけ,1game,None,2023-07-23 07:39:16.887209
3,100,来店実戦_BASHtv,BASHtv,None,2023-07-23 07:39:17.411343
4,100,自由打夢,BASHtv,None,2023-07-23 07:39:17.924061
...,...,...,...,...,...
742,0,Bakyun!スケールL,パチスロ必勝本,None,2023-08-14 07:34:58.616657
743,0,スロッター×スロッター30,パチンコ店長のホール攻略,None,2023-08-14 07:34:59.140888
744,0,アルティメットちゅんげー,天草ヤスヲ,None,2023-08-14 07:35:00.775477
745,0,裏寿司(かっぱ巻き),裏寿司解析,None,2023-08-14 07:35:01.330241


In [282]:
concat_pledge_df = pd.concat([read_spreadsheet_syuzai_df,pledge_table_df],axis=0)
concat_pledge_df

,syuzai_name,baitai_name,kouyaku,id,取得時間
0,金色バナー,裏店長ミリオネア,・スロットコーナーで10.5割以上の営業を実施? ・3台以上構成の2機種以上に全台設定⑤⑥を投入?,NaN,NaT
1,虹色バナー,裏店長ミリオネア,・3台以上構成の2機種以上に全台設定⑤⑥を投入?・4台以上機種の1台以上に設定⑥投入? ・バ...,NaN,NaT
2,蛇色バナー,裏店長ミリオネア,・20円スロットの10％以上に設定⑤⑥投入? ・5台以上並びの1ヶ所以上に設定⑤⑥投入? 各...,NaN,NaT
3,黒色バナー,裏店長ミリオネア,・20円スロットの6％以上に設定⑤⑥投入? ・3台以上並びの1ヶ所以上に設定⑤⑥投入?,NaN,NaT
4,緑色バナー,裏店長ミリオネア,3台以上機種の1機種以上に設定⑤⑥投入?,NaN,NaT
...,...,...,...,...,...
742,Bakyun!スケールL,パチスロ必勝本,None,0.0,2023-08-14 07:34:58.616657
743,スロッター×スロッター30,パチンコ店長のホール攻略,None,0.0,2023-08-14 07:34:59.140888
744,アルティメットちゅんげー,天草ヤスヲ,None,0.0,2023-08-14 07:35:00.775477
745,裏寿司(かっぱ巻き),裏寿司解析,None,0.0,2023-08-14 07:35:01.330241


In [309]:
non_duplicated_concat_pledge_df = concat_pledge_df[~concat_pledge_df.duplicated(subset='syuzai_name' ,keep=False)].sort_values('syuzai_name')

In [310]:
non_duplicated_concat_pledge_df

,syuzai_name,baitai_name,kouyaku,id,取得時間
2,1GAMEホール取材 むつかけ,1game,None,100.0,2023-07-23 07:39:16.887209
1,1GAMEホール取材 オブジョイトイ,1game,None,100.0,2023-07-23 07:39:16.372895
4129,1日回胴録ハンチョウ石神収録 ガット石神,1GAME,来店実践,NaN,NaT
715,1日回胴録ハンチョウ石神収録(ガット石神),1game,None,0.0,2023-08-13 07:35:40.299515
2935,2台並びドキュメントレポート,爆ガチ!,2台構成機種の1/3以上に設定④⑤⑥投入,NaN,NaT
...,...,...,...,...,...
2923,Ａタイプアタック‼,爆ガチ!,Aタイプが対象 設定⑥を何ヶ所かに,NaN,NaT
3910,Ａレポート,アツ姫,Aタイプに高設定投入,NaN,NaT
2945,Ｒｅ：パチから始める鬼レポート,爆ガチ!,『Re:ゼロから始める異世界生活 鬼がかりver』が対象?,NaN,NaT
563,ＺＺスロッター,パチンコ店長のホール攻略,None,0.0,2023-08-08 19:58:21.218513


In [290]:
extract_df

,syuzai_name,baitai_name,kouyaku,id,取得時間
630,ｙｔｒ来店実践,セブンズTV,None,0.0,2023-08-09 07:35:02.578280
3964,ｙｔｒ来店実践,SEVENSTV,来店実践,NaN,NaT


In [289]:
record_df = pd.DataFrame([[]])

,syuzai_name,baitai_name,kouyaku,id,取得時間
630,ｙｔｒ来店実践,セブンズTV,None,0.0,2023-08-09 07:35:02.578280
3964,ｙｔｒ来店実践,SEVENSTV,来店実践,NaN,NaT


In [292]:
new_syuzai_name = extract_df.iloc[0, 0]
print(new_syuzai_name)
new_media_name = extract_df.iloc[0, 1]
print(new_media_name)
new_please_name = extract_df.iloc[-1, 2]
print(new_please_name)

ｙｔｒ来店実践
セブンズTV
来店実践


In [ ]:
syuzai_write_df = pd.DataFrame(columns=['syuzai_name','baitai_name','kouyaku'])
for syuzai_name in duplicated_concat_pledge_df['syuzai_name'].unique():
    extract_df = duplicated_concat_pledge_df[duplicated_concat_pledge_df['syuzai_name'] == syuzai_name]
    #display(extract_df.sort_values('取得時間'))
    extract_df=extract_df.sort_values('取得時間')
    new_syuzai_name = extract_df.iloc[0, 0]
    #print(new_syuzai_name)
    new_media_name = extract_df.iloc[0, 1]
    #print(new_media_name)
    new_pledge_name = extract_df.iloc[-1, 2]
    #print(new_please_name)
    record_df = pd.DataFrame([[new_syuzai_name, new_media_name, new_pledge_name ]], columns=['syuzai_name','baitai_name','kouyaku'])
    syuzai_write_df = pd.concat([syuzai_write_df, record_df], axis=0)
    display(record_df)
    #print('------------------------')


In [307]:
syuzai_write_df = syuzai_write_df[syuzai_write_df['syuzai_name'].str.contains('広告') == False]
syuzai_write_df

,syuzai_name,baitai_name,kouyaku
0,6.5分析,ななライズ,2台以上設置の6.5号機の1機種以上が全台設定⑤⑥ 最低1台以上は設定⑥
0,Aプロジェクト,ジャンバリ.TV,Aタイプコーナー対象
0,BURST,スクープTV,設定⑤⑥が5台並び 設置2箇所以上
0,Bakyun!,パチスロ必勝本,５台以上並びで設定６が\n ２ヵ所以上設置(台数で変化)
0,Bakyun!スケールL,パチスロ必勝本,5台並びで7〜8箇所が全④⑤⑥
...,...,...,...
0,降臨シリーズ シヴァ,DMMぱちタウン,設定６を６機種に投入
0,降臨シリーズ テミス,DMMぱちタウン,・20円スロットの6.5号機の15％以上に設定⑤⑥を投入? ・設定⑥の割合は50％以上? 設...
0,降臨シリーズ トールハンマー,DMMぱちタウン,特定末尾の1/2に設定⑥投入? 残りの1/2には設定④⑤⑥投入?
0,降臨シリーズ ヘラクレス,DMMぱちタウン,スロットコーナー全体で10割以上の営業?


In [ ]:
extract_df

In [ ]:
concat_pledge_df

In [279]:
concat_pledge_df.to_csv('csv/test_pledge.csv', index=False)

In [216]:
set(kouyaku_df_type_2['media_name'].unique()) - set(set(pledge_table_df['媒体名'].unique()) & set(kouyaku_df_type_2['media_name'].unique()))

{'Aims (エイムズ)',
 'BASHtv(バッシュTV)',
 'Gooパチ',
 'HAZUSE',
 'PACHINKO&SLOT アナコンダ',
 'ScooP!tv',
 'あかだ主任のガチンコホール',
 'じゃんじゃん',
 'そにま(Sonic Map)',
 'でちゃう!',
 'アツモリ！',
 'ウルフスロット',
 'オフミー(パチミー)',
 'ゲッツ!!',
 'シバター来店',
 'スパ満開',
 'スロット',
 'スロットスター',
 'スロットライブ【RED BLUE】',
 'ズバット!!',
 'ゼロノス',
 'タロット',
 'パチンコ',
 'パチンコウォーカー',
 'ヒッポ',
 'モリピ',
 'リアルガチなホールチェック',
 '一撃',
 '並ばせ屋山本',
 '五神星',
 '共闘戦線(RISING OUT)(エースプロ)',
 '娯楽の殿堂',
 '寿司投げ',
 '日雇いホールアドバイザー',
 '爆ガチ！',
 '肉はこう焼く!(エースプロ)',
 '虹なな(エースプロ)',
 '野菜はこう焼く！',
 '鉄カバ(ヒッポ)',
 '飛蔵(とぶぞう)のパチンコ道場',
 '鬼姫'}

In [ ]:
read_spreadsheet_syuzai_df

In [311]:
non_duplicated_concat_pledge_df

,syuzai_name,baitai_name,kouyaku,id,取得時間
2,1GAMEホール取材 むつかけ,1game,None,100.0,2023-07-23 07:39:16.887209
1,1GAMEホール取材 オブジョイトイ,1game,None,100.0,2023-07-23 07:39:16.372895
4129,1日回胴録ハンチョウ石神収録 ガット石神,1GAME,来店実践,NaN,NaT
715,1日回胴録ハンチョウ石神収録(ガット石神),1game,None,0.0,2023-08-13 07:35:40.299515
2935,2台並びドキュメントレポート,爆ガチ!,2台構成機種の1/3以上に設定④⑤⑥投入,NaN,NaT
...,...,...,...,...,...
2923,Ａタイプアタック‼,爆ガチ!,Aタイプが対象 設定⑥を何ヶ所かに,NaN,NaT
3910,Ａレポート,アツ姫,Aタイプに高設定投入,NaN,NaT
2945,Ｒｅ：パチから始める鬼レポート,爆ガチ!,『Re:ゼロから始める異世界生活 鬼がかりver』が対象?,NaN,NaT
563,ＺＺスロッター,パチンコ店長のホール攻略,None,0.0,2023-08-08 19:58:21.218513


In [323]:
write_concat_df = pd.concat([syuzai_write_df,non_duplicated_concat_pledge_df],axis=0)
write_concat_df  = write_concat_df [['syuzai_name','baitai_name','kouyaku']]
write_concat_df['created_at'] = datetime.datetime.now() + datetime.timedelta(hours=9)
write_concat_df = write_concat_df[(write_concat_df['kouyaku'].isnull() != True) & (write_concat_df['kouyaku'] != '')]
write_concat_df

,syuzai_name,baitai_name,kouyaku,created_at
0,6.5分析,ななライズ,2台以上設置の6.5号機の1機種以上が全台設定⑤⑥ 最低1台以上は設定⑥,2023-08-15 11:12:54.901850
0,Aプロジェクト,ジャンバリ.TV,Aタイプコーナー対象,2023-08-15 11:12:54.901850
0,BURST,スクープTV,設定⑤⑥が5台並び 設置2箇所以上,2023-08-15 11:12:54.901850
0,Bakyun!,パチスロ必勝本,５台以上並びで設定６が\n ２ヵ所以上設置(台数で変化),2023-08-15 11:12:54.901850
0,Bakyun!スケールL,パチスロ必勝本,5台並びで7〜8箇所が全④⑤⑥,2023-08-15 11:12:54.901850
...,...,...,...,...
3392,龍の子太郎,ゼロノス,各島に設定⑥投入,2023-08-15 11:12:54.901850
2911,１０スロの真髄レポート,爆ガチ!,10スロが対象 公約はナシ,2023-08-15 11:12:54.901850
2923,Ａタイプアタック‼,爆ガチ!,Aタイプが対象 設定⑥を何ヶ所かに,2023-08-15 11:12:54.901850
3910,Ａレポート,アツ姫,Aタイプに高設定投入,2023-08-15 11:12:54.901850


In [326]:
write_concat_df

,syuzai_name,baitai_name,kouyaku,created_at
0,6.5分析,ななライズ,2台以上設置の6.5号機の1機種以上が全台設定⑤⑥ 最低1台以上は設定⑥,2023-08-15 11:12:54.901850
0,Aプロジェクト,ジャンバリ.TV,Aタイプコーナー対象,2023-08-15 11:12:54.901850
0,BURST,スクープTV,設定⑤⑥が5台並び 設置2箇所以上,2023-08-15 11:12:54.901850
0,Bakyun!,パチスロ必勝本,５台以上並びで設定６が\n ２ヵ所以上設置(台数で変化),2023-08-15 11:12:54.901850
0,Bakyun!スケールL,パチスロ必勝本,5台並びで7〜8箇所が全④⑤⑥,2023-08-15 11:12:54.901850
...,...,...,...,...
3392,龍の子太郎,ゼロノス,各島に設定⑥投入,2023-08-15 11:12:54.901850
2911,１０スロの真髄レポート,爆ガチ!,10スロが対象 公約はナシ,2023-08-15 11:12:54.901850
2923,Ａタイプアタック‼,爆ガチ!,Aタイプが対象 設定⑥を何ヶ所かに,2023-08-15 11:12:54.901850
3910,Ａレポート,アツ姫,Aタイプに高設定投入,2023-08-15 11:12:54.901850


In [370]:
from selenium import webdriver
import unicodedata
import time
import pandas as pd
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import requests
import json
from webdriver_manager.chrome import ChromeDriverManager
import mysql
import mysql.connector
import os
import datetime
import psycopg2
from dotenv import load_dotenv
load_dotenv(".env")
print('ライブラリの読み込み完了')
def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 

def login_scraping_site(area_name):
    options = Options()
    options.add_argument('--headless')
    options.add_argument("--no-sandbox")

    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)#ChromeDriverManager().install() 
    browser.implicitly_wait(10)
    url_login = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/login_form_mail"
    #admageを開く
    browser.get(url_login)
    browser.implicitly_wait(10)

    # id
    element = browser.find_element(By.NAME,"id")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_ID'))

    # pw
    element = browser.find_element(By.NAME,"pass")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_PW'))

    browser.implicitly_wait(10)
    element = browser.find_element(By.CLASS_NAME,"box_hole_view_report_input")
    element.click()
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/select_area"
    browser.get(url)
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/?area={area_name}"
    browser.get(url)
    browser.implicitly_wait(10)
    return browser



users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
port = 5432        # DBが稼働しているポート番号(適宜変更)

# PostgreSQLへ接続
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

# PostgreSQLにデータ登録
cursor = conn.cursor()
sql = f"""
        SELECT *
        FROM pledge
        """
print(sql)
cursor.execute(sql)
#cols = [col[0] for col in cursor.description]
sql_syuzai_report_all_df = pd.DataFrame(cursor.fetchall(),columns = ['id','syuzai_name','media_name','pledge_text','created_at'])
max(list(sql_syuzai_report_all_df['id']))


ライブラリの読み込み完了

        SELECT *
        FROM pledge
        


,id,syuzai_name,media_name,pledge_text,created_at
0,14,6.5分析,ななライズ,2台以上設置の6.5号機の1機種以上が全台設定⑤⑥ 最低1台以上は設定⑥,2023-08-15 03:32:40.466122
1,15,Aプロジェクト,ジャンバリ.TV,Aタイプコーナー対象,2023-08-15 03:32:40.830257
2,16,BURST,スクープTV,設定⑤⑥が5台並び 設置2箇所以上,2023-08-15 03:32:41.349617
3,17,Bakyun!,パチスロ必勝本,５台以上並びで設定６が\n ２ヵ所以上設置(台数で変化),2023-08-15 03:32:41.862476
4,18,Bakyun!スケールL,パチスロ必勝本,5台並びで7〜8箇所が全④⑤⑥,2023-08-15 03:32:42.372501
...,...,...,...,...,...
1146,1160,龍の子太郎,ゼロノス,各島に設定⑥投入,2023-08-15 03:42:48.177938
1147,1161,１０スロの真髄レポート,爆ガチ!,10スロが対象 公約はナシ,2023-08-15 03:42:48.687498
1148,1162,Ａタイプアタック‼,爆ガチ!,Aタイプが対象 設定⑥を何ヶ所かに,2023-08-15 03:42:49.207470
1149,1163,Ａレポート,アツ姫,Aタイプに高設定投入,2023-08-15 03:42:49.707205


In [371]:
increment_id = max(list(sql_syuzai_report_all_df['id']))

1164

In [369]:
increment_id = max(list(sql_syuzai_report_all_df['id']))
for i,row in write_concat_df.iterrows():
    #print(row)
    increment_id += 1
    try:
        cursor.execute("INSERT INTO pledge VALUES (%s,%s,%s, %s, %s)", (increment_id ,row['syuzai_name'],row['baitai_name'],row['kouyaku'],datetime.datetime.now()))
        conn.commit()
    except:
        conn.rollback()
        continue


# post_line_text(text,os.getenv('LINE_TOKEN'))